# Explainable Boosting Machine

Explainable Boosting Machine (EBM) はtree-basedの一般化加法モデル（generalized additive model: GAM）で、

- 交互作用の自動検知
- 解釈性の高さ

が特徴

## アルゴリズム

EBMはboostingないし一般化加法モデル（GAM）の形式をとる：

$$
g(E[y])=\beta_0+\sum f_i(x_i)+\sum f_{i, j}(x_i, x_j)
$$

ここで$g()$は回帰や分類の設定にGAMを適用するためのlink function。

EBMは伝統的なGAMからの進化点として

1. 特徴関数（feature function）$f_j$にbaggingや勾配ブースティングを使用する
2. 交互作用項 $f_{i,j}(x_i,x_j)$ の自動検知ができる





## 統計的推論

[[2601.18857v1] Statistical Inference for Explainable Boosting Machines](https://arxiv.org/abs/2601.18857v1)

## 実装

[Explainable Boosting Machine — InterpretML documentation](https://interpret.ml/docs/ebm.html)

In [4]:
import pandas as pd
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error
from interpret.glassbox import ExplainableBoostingRegressor
from interpret import show

# 1) データロード
data = fetch_california_housing(as_frame=True)
X: pd.DataFrame = data.data
y: pd.Series = data.target  # Median house value

# 2) train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 3) EBM（回帰）
ebm = ExplainableBoostingRegressor(
    random_state=42,
    # interactions=10,   # 交互作用を入れたいなら（解釈はやや複雑に）
    # max_bins=256,      # 連続特徴のビン数（大きいほど柔軟だが過学習注意）
)

# 4) fit & 評価
ebm.fit(X_train, y_train)

pred = ebm.predict(X_test)
print(f"R^2: {r2_score(y_test, pred):.3f}")
print(f"MAE: {mean_absolute_error(y_test, pred):.3f}")

# 5) Explain（global / local）
global_exp = ebm.explain_global(name="EBM global (California Housing)")
local_exp = ebm.explain_local(X_test.iloc[:5], y_test.iloc[:5], name="EBM local (first 5)")

# Notebook環境なら可視化（ブラウザ/埋め込み表示）
show(global_exp)
show(local_exp)


R^2: 0.841
MAE: 0.307


<!-- http://127.0.0.1:7001/129812747188736/ -->

<!-- http://127.0.0.1:7001/129812617362352/ -->

## 参考文献

- [Lou, Y., Caruana, R., Gehrke, J., & Hooker, G. (2013, August). Accurate intelligible models with pairwise interactions. In *Proceedings of the 19th ACM SIGKDD international conference on Knowledge discovery and data mining* (pp. 623-631).](https://www.cs.cornell.edu/~yinlou/papers/lou-kdd13.pdf)